<h2><center> Scripts AMR to DG mapping </center></h2>

### General imports:

In [1]:
import pandas as pd
from pathlib import Path 

### 1. Generating AMR from corpus: 

#### 1.1 Loading corpus:

In [2]:
sts_path = Path('.')/'datasets'/'sts'
sts = pd.read_csv(sts_path/'STS2016_full.tsv', header=0, sep='\t', usecols=list(range(2)))

sents = [sts['sent1'].tolist(), sts['sent2'].tolist()]

#print(sts.info())
print(sents[0][0], sents[1][0])
sts.head(2)

Should I use IRA money to pay down my student loans? Should I cash out my IRA to pay my student loans?


,sent1,sent2
0,Should I use IRA money to pay down my student ...,Should I cash out my IRA to pay my student loans?
1,How do I keep an egg from cracking while being...,How do I prevent an egg cracking while hard bo...


#### 1.2 Running an Text2AMR-parser:

In [3]:
import amrlib 
amrlib.setup_spacy_extension()

stog = amrlib.load_stog_model()
sents1_amr = stog.parse_sents(sents[0], add_metadata=True)
sents2_amr = stog.parse_sents(sents[1], add_metadata=True)

def save_amrs(amrs, savepath):
    with open(sts_path/savepath, 'w') as f:
        for amr in amrs:
            print(amr, file=f, end='\n\n')

save_amrs(sents1_amr, 'STS2016_sents1_AMR.txt')
save_amrs(sents2_amr, 'STS2016_sents2_AMR.txt')

Loading model C:\ProgramData\Anaconda3\envs\dlp38\Lib\site-packages\amrlib\data\model_stog\model.pt


### 2. Generating UD-DG from corpus: 

In [16]:
from supar import Parser
import spacy; nlp = spacy.load('en_core_web_sm')

parser = Parser.load('crfnp-dep-en')

docs1 = list(nlp.pipe(sents[0]))
docs2 = list(nlp.pipe(sents[1]))

toks1 = [[tok.text for tok in doc] for doc in docs1]
toks2 = [[tok.text for tok in doc] for doc in docs2]

dataset1 = parser.predict(toks1, prob=True, verbose=False)
dataset2 = parser.predict(toks2, prob=True, verbose=False)
dg_parses1, dg_parses2  = dataset1.sentences, dataset2.sentences

def save_dg(dgs, savepath):
    with open(sts_path/savepath, 'w') as f:
        for dg in dgs:
            print(dg, file=f, end='\n')

save_dg(dg_parses1, 'STS2016_sents1_DG.txt')
save_dg(dg_parses2, 'STS2016_sents2_DG.txt')
#for parse in dg_parses1[:3]: print(parse)

100%|####################################| 33/33 00:07<00:00,  4.14it/s            
100%|####################################| 31/31 00:07<00:00,  3.91it/s            


### 3. Running Szubert et al. (2018) scripts:

In [4]:
# Steps necessary to run those scripts (admin rights required):
# pip install num2words
# pip install Morfessor
# pip install pycorenlp
!conda activate dlp38 && python szubert_parser_py3\align.py -p "datasets/sts/STS2016_sents1_DG.txt" -a "datasets/sts/STS2016_sents1_AMR.txt" -o "STS2016_sents1_AMR2DG.txt"

Traceback (most recent call last):
  File "szubert_parser_py3\align.py", line 18, in <module>
    sentences = read_in_fixed_parses(read(opts.amr_file), opts.ud_file)
  File "C:\Users\Denis Logvinenko\Documents\Projects\SWP\szubert_parser_py3\parse.py", line 30, in read
    return process_amr_input_file(amr_file, max_sent_length, parse=False, isi=isi, jamr=jamr)
  File "C:\Users\Denis Logvinenko\Documents\Projects\SWP\szubert_parser_py3\parse.py", line 66, in process_amr_input_file
    amr_concepts, amr2jamr, amr2isi, amr_relations = process_amr(full_amr, jamr=jamr, isi=isi)
  File "C:\Users\Denis Logvinenko\Documents\Projects\SWP\szubert_parser_py3\parse.py", line 176, in process_amr
    concept_dict, graph = parse_amr(amr_string, concept_dict, [], {}, [], 0)
TypeError: cannot unpack non-iterable NoneType object
